Mount to Google Drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

MessageError: ignored

Definimos el base path donde guardaremos toda nuestra informacion relacionada al reto.

In [ ]:
basePath = '/content/gdrive/Shareddrives/TC3007C Inteligencia Artificial'
dataPath = f"/{basePath}/Data"

## Utils

In [ ]:
import zipfile
import shutil
import pathlib
import pandas as pd
import os
import subprocess
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import cv2

In [ ]:
def count_files_in_subd(path):
    filescount = {}
    for root, dirs, files in os.walk(path):
        filescount[root.split("/")[-1]] = len(files)
    return pd.DataFrame.from_dict(filescount.items(), orient='columns')

def balance_data(df, target_size=12):
    """
    Increase the number of samples to number_of_samples for every label

        Example:
        Current size of the label a: 10
        Target size: 23

        repeat, mod = divmod(target_size,current_size) 
        2, 3 = divmod(23,10)

        Target size: current size * repeat + mod 

    Repeat this example for every label in the dataset.
    """

    df_groups = df.groupby(['label'])
    df_balanced = pd.DataFrame({key:[] for key in df.keys()})

    for i in df_groups.groups.keys():
        df_group = df_groups.get_group(i)
        df_label = df_group.sample(frac=1)
        current_size = len(df_label)

        if current_size >= target_size:
            # If current size is big enough, do nothing
            pass
        else:

            # Repeat the current dataset if it is smaller than target_size 
            repeat, mod = divmod(target_size,current_size)
            df_label_new = pd.concat([df_label]*repeat,ignore_index=True,axis=0)
            df_label_remainder = df_group.sample(n=mod)
            df_label_new = pd.concat([df_label_new,df_label_remainder],ignore_index=True,axis=0)
            # print(df_label_new)
        df_balanced = pd.concat([df_balanced,df_label_new],ignore_index=True,axis=0)
    return df_balanced    

# Original Dataset

Instalamos la libreria Kaggle utilizando pip para descargar de la api los datos del reto. Pueden consultar el siguiente link: https://www.kaggle.com/docs/api para mas informacion de la api de Kaggle 

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Definimos el path en donde se encuentra nuestro .kaggle.json donde tenemos nuestras credenciales para acceder a la api de Kaggle

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

Nos movemos a donde queremos descargar el zip con los datos de Kaggle

In [ ]:
%cd "/content/gdrive/Shareddrives/TC3007C Inteligencia Artificial/Data"
%ls

/content/gdrive/Shareddrives/TC3007C Inteligencia Artificial/Data
data_15/  data_30/  data_5/  data_original/


Corremos el comando !kaggle competitions download -c "nombre de la competencia", el cual descargara un .zip con el dataset de la competencia. Despues procedemos a hacer unzip de los datos.

In [ ]:
file = pathlib.Path("hotel-id-to-combat-human-trafficking-2022-fgvc9.zip")
if file.exists():
    print("File exist")
else:
    print("File not exist")
    !kaggle competitions download -c hotel-id-to-combat-human-trafficking-2022-fgvc9
    if file.exists():
        # Unzip
        for file in os.listdir():
          if file.endswith(".zip"):
              with zipfile.ZipFile(file, "r") as zip_file:
                  zip_file.extractall()
              os.remove(file)

Hacemos uso de la funcion helper count_files_in_subd la cual retorna un dataset con los nombres de las clases y el numero de imagenes en cada clase.

In [ ]:
filesCount = count_files_in_subd('/content/gdrive/Shareddrives/TC3007C Inteligencia Artificial/Data/data_15')
filesCount.head()

In [ ]:
filesCount.describe()

In [ ]:
print((filesCount[1] == 0).sum())
print((filesCount[1] < 5).sum())
print((filesCount[1] < 15).sum())
print((filesCount[1] < 30).sum())

# Split


En split_by definimos el split que gustariamos hacer de nuestro dataset, en este caso lo definimos como 10, despues procedemos a crear las carpetas donde se guardaran los datasets de train, validation y train_augmentation.

In [ ]:
# Define desired split.
split_by = 15
# Define data path.
dataPath = f"{basePath}/Data"
splitDataPath = os.path.join(dataPath, f"data_{split_by}")

try:
    os.mkdir(splitDataPath)
except OSError as error:
    print(error)  

newTrainPath = os.path.join(splitDataPath, "train")
newTestPath = os.path.join(splitDataPath, "validation")
newAugmentationPath = os.path.join(splitDataPath, "train_augmentation")

try:
    os.mkdir(newTrainPath)
    os.mkdir(newTestPath)
    os.mkdir(newAugmentationPath)
except OSError as error:
    print(error)    

[Errno 17] File exists: '/content/gdrive/Shareddrives/TC3007C Inteligencia Artificial/Data/data_15'


Procedemos a hacer el split a partir del dataset original, en este caso haciendo un split del 0.9 definido en la variable 0.9

In [ ]:
# Define split percentage.
split_percent = 0.9
for root, dirs, files in os.walk(f'{dataPath}/data_original'):
    fileLength = len(files)
    # Depending on the split_by defined, we will be discarting folders with less images than defined by split_by.
    if(fileLength >= split_by):
        className = root.split("/")[-1]
        trainingFiles = files[0: int(fileLength * split_percent)]
        testFiles = files[int(fileLength * split_percent):]
        print(trainingFiles)
        print(testFiles)
        # Proceed to create folder and copy into that folder images on first split.
        for imageFile in trainingFiles:
          imageFolderPath = os.path.join(newTrainPath, className)
          file = pathlib.Path(imageFolderPath)
          if file.exists():
            pass
          else:
            os.mkdir(imageFolderPath)
          shutil.copy(os.path.join(root, imageFile), imageFolderPath)
        # Proceed to create folder and copy into that folder images on second split.
        for imageFile in testFiles:
          imageFolderPath = os.path.join(newTestPath, className)
          file = pathlib.Path(imageFolderPath)
          if file.exists():
            pass
          else:
            os.mkdir(imageFolderPath)
          shutil.copy(os.path.join(root, imageFile), imageFolderPath)

Corremos las funciones helper para obtener el numero de imagenes en cada clase.

In [ ]:
count_files_in_subd(newTrainPath)

In [ ]:
count_files_in_subd(newTestPath)

# Balanced And Augmented Dataset
The original dataset was amputed to only include classes that had at least 15 images. Those 15 images were used to oversample each class to reach at least 100 images per class, the classes that had 100 and more weren't amputated.


In [ ]:
data = []
root_path = splitDataPath + "/train"
for folder in sorted(os.listdir(root_path)):
    folder_path =  root_path+"/"+folder
    for file in sorted(os.listdir(folder_path)):
        file_path = folder_path+"/"+file
        data.append((folder, file_path))

df = pd.DataFrame(data, columns=['label', 'img_path'])
df

,label,img_path
0,100206,/content/gdrive/Shareddrives/TC3007C Inteligen...
1,100206,/content/gdrive/Shareddrives/TC3007C Inteligen...
2,100206,/content/gdrive/Shareddrives/TC3007C Inteligen...
3,100206,/content/gdrive/Shareddrives/TC3007C Inteligen...
4,100206,/content/gdrive/Shareddrives/TC3007C Inteligen...
...,...,...
18298,81786,/content/gdrive/Shareddrives/TC3007C Inteligen...
18299,81786,/content/gdrive/Shareddrives/TC3007C Inteligen...
18300,81786,/content/gdrive/Shareddrives/TC3007C Inteligen...
18301,81786,/content/gdrive/Shareddrives/TC3007C Inteligen...


In [ ]:
df_balanced = balance_data(df, 100)

In [ ]:
for label in df_balanced.label.unique():
    # A particular image will be transformed to its various versions within the augmentation step 
    image_datagen = ImageDataGenerator(
    rotation_range=130,
    width_shift_range=0.1,
    height_shift_range=0.15,
    brightness_range=(0.2,1),
    channel_shift_range=150,
    shear_range = 0.2,
    zoom_range = -0.2, 
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode = 'reflect', cval = 125)

    augm_folder_path = newAugmentationPath+"/"+label
    os.mkdir(augm_folder_path)

    # "augment" folder should already exist
    image_generator = image_datagen.flow_from_dataframe(
                dataframe=df_balanced[df_balanced["label"] == label],
                x_col="img_path",
                y_col="label",
                class_mode="categorical",
                batch_size=32,  # could be 4 or 16
                save_to_dir=augm_folder_path,
                shuffle=True 
                )
    for i in range(len(image_generator)):
        image_generator.next()

Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 200 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belonging to 1 classes.
Found 100 validated image filenames belo

In [ ]:
count_files_in_subd(newAugmentationPath)